In [68]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
def read_CBE_data(path, page=0):
    data = pd.read_excel(path, sheet_name=page)
    
    data = data[[
    "IP",
    "DISTRICT",
    "COMMUNITY/CLASS",
    "TYPE OF OOSC",
    "SEX",
    "AGE",
    "DROP OUT (YES/NO)",
    "GRADUATED (YES/NO)",
    "Language"]]
    
    data["DROP OUT (YES/NO)"] = data["DROP OUT (YES/NO)"].replace(["NO", "No"], "NO")
    data["DROP OUT (YES/NO)"] = data["DROP OUT (YES/NO)"].replace(["YES", "Yes"], "YES")
    
    return data

In [72]:
cycle_1 = read_CBE_data("shared_data_read_only/Data/AfriKids/InternalData/AfriKids_CBE_Data/AfriKids CBE Cycle 1 Learner Facilitator LC-SMC Training.xls", "Learner Data Template")
cycle_2 = read_CBE_data("shared_data_read_only/Data/AfriKids/InternalData/AfriKids_CBE_Data/AfriKids CBE Cycle 2 Learner Facilitator LC-SMC Training.xls", "Learner Data Template")
cycle_3 = read_CBE_data("shared_data_read_only/Data/AfriKids/InternalData/AfriKids_CBE_Data/AfriKids CBE Cycle 3 Learner Facilitator LC-SMC Training.xlsx", "Learner Data Template")
cycle_4 = read_CBE_data("shared_data_read_only/Data/AfriKids/InternalData/AfriKids_CBE_Data/AfriKids CBE Cycle 4 Learner Facilitator LC-SMC Training.xls", "Learner Data Template")
cycle_5 = read_CBE_data("shared_data_read_only/Data/AfriKids/InternalData/AfriKids_CBE_Data/Afrikids CBE Cycle 5  Learner Facilitator LC-SMC Training.xls")

In [73]:
all_results = pd.concat([cycle_1, cycle_2, cycle_3, cycle_4, cycle_5])